In [295]:
%pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import talib
from talib import MA_Type
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import backtesting
from backtesting import Backtest, Strategy
import optuna

In [3]:
# Загрузка данных из CSV файла, который мы получили в ходе выполнения первого ДЗ
def LoadAndFill(fileName: str) -> pd.DataFrame:
    df = pd.read_csv(fileName, header=[0, 1], index_col=0)
    df.columns.names = ['Price', 'Date'] # Переименовываем индексное поле из Ticker в Date
    df.index = pd.to_datetime(df.index)
    expected_dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    # missing_dates = expected_dates.difference(df.index)
    df = df.reindex(expected_dates)
    df.ffill(inplace=True)
    return df

# Нормализация данных к диапазону 0..1
def ScaleDF(adj_close_df: pd.DataFrame) -> pd.DataFrame:
    scaler = MinMaxScaler()
    scaled_adj_close = pd.DataFrame(
        scaler.fit_transform(adj_close_df),
        columns=adj_close_df.columns,
        index=adj_close_df.index
    )
    return scaled_adj_close

# Получение цены закрытия и объема для конкретного актива из нащего общего DF
def GetClosePriceAndVolumeForAsset(name: str, close_price_df: pd.DataFrame, volume_norm: pd.DataFrame) -> pd.DataFrame:  
    close_price_df = close_price_df[name].to_frame()
    close_price_df.rename(columns={name: 'Close'}, inplace=True)
    close_price_df.rename(columns={'index': 'Date'}, inplace=True)

    volume_norm = volume_norm[name].to_frame()
    volume_norm.rename(columns={name: 'Volume'}, inplace=True)
    volume_norm.rename(columns={'index': 'Date'}, inplace=True)

    close_price_df['Volume'] = volume_norm['Volume']
    return close_price_df
    
    
def PlotPrices(df: pd.DataFrame):
    # Plot the scaled data
    df.plot(figsize=(12, 6), title="Нормализованная скорректированная цена закрытия")
    plt.xlabel("Дата")
    plt.ylabel("Цена от 0 до 1")
    plt.grid(True)
    plt.legend(title="Stocks", loc="upper left")
    plt.show()

In [4]:
# Расчета веса сигнала. Если сигнал нулевой, то вес затухает на половину веса от предыдущего дня, если сигнал положительный +1 или отрицательный -1, то он прибавляется к весу
def add_signal_weight(signal_df: pd.DataFrame):
    prev_sig_weight = 0.0
    signal_df['singal_weight'] = 0.0
    for row in signal_df.itertuples(index=True, name='Row'):      
        if row.Signal != 0:         
            # print(apple_df_signals.iloc[row.Index]['singal weight'])      
            signal_df['singal_weight'].iloc[row.Index] = prev_sig_weight + row.Signal                
            # print(row.Index, row.Signal, prev_sig_weight, prev_sig_weight + row.Signal, apple_df_signals['singal_weight'].iloc[row.Index])      
        else:
            signal_df['singal_weight'].iloc[row.Index] = prev_sig_weight / 2.0
        prev_sig_weight = signal_df['singal_weight'].iloc[row.Index]
    signal_df['singal_weight'] = ScaleDF(signal_df['singal_weight'].to_frame()) - 0.5
    # print(row)


In [5]:
# Расчет сигнала на основе линий Боллинджера 
def bbands_signals(input_df: pd.DataFrame) -> pd.DataFrame:    
    period = 20
    upper, middle, lower = talib.BBANDS(input_df['Close'], matype=MA_Type.SMA, timeperiod=period)
    input_df['upper'], input_df['middle'], input_df['lower'] = upper.astype('float64'), middle.astype('float64'), lower.astype('float64')
    input_df = input_df.iloc[period-1:]
    input_df['Prev Close'] = input_df['Close'].shift(1)

    signal_df = pd.DataFrame(index=input_df.index)
    signal_df['Signal'] = 0
    buy_condition = (input_df['Prev Close'] >= input_df['lower']) & (input_df['Close'] < input_df['lower'])
    sell_condition = (input_df['Prev Close'] <= input_df['upper']) & (input_df['Close'] > input_df['upper'])
    signal_df.dropna(inplace=True)
    assert signal_df.index.equals(input_df.index), "Indices do not match"

    buy_condition.fillna(False)
    sell_condition.fillna(False)
    signal_df.loc[buy_condition, 'Signal'] = 1   # Buy signal
    signal_df.loc[sell_condition, 'Signal'] = -1  # Sell signal
    signal_df['Close'] = input_df['Close']
    signal_df['Volume'] = input_df['Volume']
    signal_df.reset_index(inplace=True)
    signal_df.rename(columns={'index': 'Date'}, inplace=True)
    add_signal_weight(signal_df)
    return signal_df


# Расчет сигнала на основе MACD
def macd_signal(input_df: pd.DataFrame) -> pd.DataFrame:    
    
    signal_df = pd.DataFrame(index=input_df.index)
    signal_df['Close'] = input_df['Close']
    signal_df['Volume'] = input_df['Volume']
    signal_df.reset_index(inplace=True)
    signal_df.rename(columns={'index': 'Date'}, inplace=True)
    signal_df['Signal'] = 0

    signal_df['tema'] = talib.TEMA(signal_df['Close'], timeperiod=24)
    signal_df['macd'], signal_df['macd_signal'], signal_df['macd_hist'] = talib.MACD(signal_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
    # Создаем сигналы для покупки и продажи
    signal_df['Signal'] = 0
    signal_df.loc[(signal_df['macd'] > signal_df['macd_signal']) & (signal_df['Close'] > signal_df['tema']), 'Signal'] = 1  # Сигнал на покупку
    signal_df.loc[(signal_df['macd'] < signal_df['macd_signal']) & (signal_df['Close'] < signal_df['tema']), 'Signal'] = -1  # Сигнал на продажу

    add_signal_weight(signal_df)
    return signal_df


def reversed_bbands_signals(input_df: pd.DataFrame) -> pd.DataFrame: 
    signals_df = bbands_signals(input_df)
    signals_df['Signal'] = signals_df['Signal'] * -1
    return signals_df

def reversed_macd_signals(input_df: pd.DataFrame) -> pd.DataFrame: 
    signals_df = macd_signal(input_df)
    signals_df['Signal'] = signals_df['Signal'] * -1
    return signals_df

In [6]:
# Отображения сигнала, объема торгов и силы сигнала
def PlotSignal(input_signal_df: pd.DataFrame):
    fig = go.Figure()

    # Create subplots
    fig = make_subplots(
        rows=3, 
        cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.05,  # Adjust space between the plots
        row_heights=[0.6, 0.2, 0.2]  # Adjust heights of the subplots
    )


    buy_signals = input_signal_df[input_signal_df['Signal'] == 1]
    sell_signals = input_signal_df[input_signal_df['Signal'] == -1]
    # Add price data
    fig.add_trace(go.Scatter(x=input_signal_df['Date'], 
                             y=input_signal_df['Close'], 
                             name='Close Price'),
                            row=1,
                            col=1)    
    
    # Add buy signals
    fig.add_trace(go.Scatter(
        x=buy_signals['Date'],
        y=buy_signals['Close'],
        mode='markers',
        name='Buy Signal',
        marker_symbol='triangle-up',
        marker_color='green',
        marker_size=10
    ),
        row=1,
        col=1)

    # Add sell signals
    fig.add_trace(go.Scatter(
        x=sell_signals['Date'],
        y=sell_signals['Close'],
        mode='markers',
        name='Sell Signal',
        marker_symbol='triangle-down',
        marker_color='red',
        marker_size=10
    ),  row=1,
        col=1)
    
    fig.add_trace(
    go.Bar(x=input_signal_df['Date'], 
           y=input_signal_df['Volume'],
           name='Volume'),
    row=2, col=1
)
    
    fig.add_trace(go.Scatter(x=input_signal_df['Date'], 
                             y=input_signal_df['singal_weight'], 
                             name='singal_weight'),
                            row=3,
                            col=1)   

    # Update layout
    fig.update_layout(
        title='Buy and Sell Signals',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        height=1000
    )

    fig.show()

In [ ]:
# Наша стратегия. Не играем на понижение! Не открывает позицию, если она уже открыта. "Играем" только на повышение
class SimpleFollowSignalsStrategy(Strategy):
    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0

    def next(self):
        # print(self.data)
        current_signal = self.signal[-1]      
        if current_signal == 1:                         
            if not self.position.is_long:
                self.buy()
                
        elif current_signal == -1:
            if self.position.is_long:
                self.position.close()
                return                          
        else:
            # Ниего не делаем, если мы уже в позиции Long
            return  

        self.previous_signal = current_signal

In [8]:
# Загружаем данные из файала из ДЗ 1.
df = LoadAndFill('../hw7/snp500_stock_data.csv')

In [10]:
# Нормализуем цену зактытия и объемы торгов к 0..1
close_price_norm = ScaleDF(df['Adj Close'])
volume_norm = ScaleDF(df['Volume'])

# Делаем две выборки, обучающую и валидационную
split_index = int(len(df) * 0.8) 

# Split the DataFrame
close_train_df = close_price_norm[:split_index]  
close_val_df = close_price_norm[split_index:]    

volume_train_df = volume_norm[:split_index]  
volume_val_df = volume_norm[split_index:]    


In [ ]:
# Эту ячейку будем использовать для визуализации реузльтата оптимзиации
# Такой же код будет использоваться в целевой функции оптимизации ниже
# params={'buy_th': 0.09117712777177613, 'sell_th': -0.2321450452084792, 'macd_coef': 0.33878133609287614, 'bband_coef': -0.8025356605004056}

# Суть стратегии заключается в том, что сначала рассчитывается сила сигнала для Bollinger и для MACD
# Чем чаще сигнал встречается, тем больше его вес. Если сигнал нулевой, то вес затухает (см add_signal_weight(...)). 
buy_th = 0.09117712777177613 # Порог после, кторого покупаем
sell_th = -0.2321450452084792  # Порог после, кторого продаем
macd_coef = 0.33878133609287614 # Влияние MACD на вес
bband_coef= -0.8025356605004056 # Влияние BBAND на вес (Забавно, что после оптимизации, влияние отрицательное)

# Берем валидационную выборку и смотрим на доходность стратгеии
# Можно проверить на любой бумаге из ['AAPL', 'GOOG','AMZN', 'MSFT', 'AMD', 'NVDA', 'IBM']

# берем цену закрытия и объем
# Объем пока никак не принимает участия в стратегии, просто задел на будущее
asset_df = GetClosePriceAndVolumeForAsset('AMZN', close_val_df, volume_val_df)

# Рассчитываем наши индикаторы и вес их сигналов
bband_signals = bbands_signals(asset_df)
macd_signals = macd_signal(asset_df)

# Суммарный вес, и влияение BBAND и MACD на суммарный вес
macd_signals['singal_weight'] =  macd_coef * macd_signals['singal_weight'] + bband_coef *  bband_signals['singal_weight']
combined_signal = macd_signals.copy()
combined_signal['Signal'] = 0.0 # Финальный сигнал
# Окончательне решение, покупть или продавать мы принимем на основе перехода суммарного веса через пороговые значения
# Суть задачи сводится к определению пороговых значение и коэф влияния индикаторов на суммарный вес
combined_signal.loc[combined_signal['singal_weight'] > buy_th, 'Signal'] = 1
combined_signal.loc[combined_signal['singal_weight'] < sell_th, 'Signal'] = -1
combined_signal['Open'], combined_signal['High'], combined_signal['Low'] = combined_signal['Close'], combined_signal['Close'], combined_signal['Close']

# Тестируем на валидационной выборке
bt = Backtest(combined_signal, SimpleFollowSignalsStrategy, cash=1, commission=.002, exclusive_orders=True)
stats = bt.run()

# Выводим статистику
print(stats)

# выводим график
bt.plot(
    plot_equity=True,
    plot_drawdown=True,
    relative_equity=False,
)

In [ ]:
# Самый нижний грфик это вес результрущего сигнала
PlotSignal(combined_signal)

In [14]:
# Автоматизируем получение коэф-ов. Для этого задаем целевую функцию и функцию расчета доходности стратегии

def calculate_return_for_asset(asset: str, 
                          buy_th: float, 
                          sell_th: float, 
                          input_close_price_norm: pd.DataFrame,
                          input_volume_norm: pd.DataFrame,
                          macd_coef = 1.0, 
                          bband_coef=1.0) -> float:
    asset_df = GetClosePriceAndVolumeForAsset(asset, input_close_price_norm, input_volume_norm)
    bband_signals = bbands_signals(asset_df)
    macd_signals = macd_signal(asset_df)
    macd_signals['singal_weight'] =  macd_coef * macd_signals['singal_weight'] + bband_coef *  bband_signals['singal_weight']
    combined_signal = macd_signals.copy()
    combined_signal['Signal'] = 0.0
    combined_signal.loc[combined_signal['singal_weight'] > buy_th, 'Signal'] = 1
    combined_signal.loc[combined_signal['singal_weight'] < sell_th, 'Signal'] = -1
    combined_signal['Open'], combined_signal['High'], combined_signal['Low'] = combined_signal['Close'], combined_signal['Close'], combined_signal['Close']
    bt = Backtest(combined_signal, SimpleFollowSignalsStrategy, cash=1, commission=.002, exclusive_orders=True)
    stats = bt.run()
    return stats['Return [%]']
    

def objective(trial):
    # Эти параметры мы оптимзируем
    buy_th = trial.suggest_float("buy_th", 0.0, 2.0)
    sell_th = trial.suggest_float("sell_th", -2.0, 0.0)
    macd_coef = trial.suggest_float("macd_coef", -1.0, 1.0)
    bband_coef = trial.suggest_float("bband_coef", -1.0, 1.0)
    
    # На этих активах мы учимся
    test_tickers = ['AAPL', 'GOOG','AMZN', 'MSFT', 'AMD', 'NVDA', 'IBM']
    ret = 0.0
    for ticker in test_tickers:
        try:
            # close_train_df, volume_train_df Это обучающие выборки
            ret += calculate_return_for_asset(ticker, buy_th, sell_th, close_train_df, volume_train_df, macd_coef, bband_coef)
        except:
            ret
    
    return ret

In [ ]:
# Проводим оптимизацию

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

In [ ]:
# Тестируем рузальтат
params = {'buy_th': 0.09117712777177613, 'sell_th': -0.2321450452084792, 'macd_coef': 0.33878133609287614, 'bband_coef': -0.8025356605004056}
returns_train = {}
returns_valid = {}
test_tickers = ['AAPL', 'GOOG','AMZN', 'MSFT', 'AMD', 'NVDA', 'IBM']    
for ticker in test_tickers:
   returns_train[ticker] = calculate_return_for_asset(ticker, params['buy_th'], params['sell_th'], close_train_df, volume_train_df, params['macd_coef'], params['bband_coef'])
   returns_valid[ticker] = calculate_return_for_asset(ticker, params['buy_th'], params['sell_th'], close_val_df, volume_val_df, params['macd_coef'], params['bband_coef'])


In [22]:
fig = go.Figure()

# Create subplots
fig = make_subplots(
    rows=2, 
    cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.05,  # Adjust space between the plots
    row_heights=[0.5, 0.5]  # Adjust heights of the subplots
)

x = list(returns_train.keys())
y = list(returns_train.values())
fig.add_trace(
    go.Bar(x=x, 
           y=y,
           name='Train dataset'),
    row=1, col=1
)

x = list(returns_valid.keys())
y = list(returns_valid.values())
fig.add_trace(
    go.Bar(x=x, 
           y=y,
           name='Validation dataset'),
    row=2, col=1
)

 # Update layout
fig.update_layout(
        title='Buy and Sell Signals',
        xaxis_title='Asset',
        yaxis_title='Returns %',
        xaxis_rangeslider_visible=False       
    )

fig.show()